In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from parsel import Selector
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# configure webdriver
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen
# configure chrome browser to not load images and javascript
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

In [2]:
import httplib2
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from parsel import Selector
from statistics import mean
import time
import requests
from datetime import datetime
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
http = httplib2.Http()
import mysql.connector

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Downloading: 100%|██████████| 6.21M/6.21M [00:05<00:00, 1.22MB/s]
C:\Users\user\AppData\Local\Temp\ipykernel_5748\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
url = "https://mi.tv/br/entrar"
driver.get(url)
driver.window_handles;
driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.CONTROL+"t");
driver.find_element(By.XPATH, '//*[@id="login-email"]').send_keys("osumorog@gmail.com")
driver.find_element(By.XPATH, '//*[@id="login-password"]').send_keys("firefire")
driver.find_element(By.XPATH, '//*[@id="btn-email-login-perform"]').click()

In [5]:
# set implicit wait time
driver.implicitly_wait(10) # seconds
driver.get("https://mi.tv/br/perfil/selecionar-canais")
page_source =driver.page_source
soup = BeautifulSoup(page_source,'html.parser')

In [6]:
buttons = soup.find("div", {"class":"container"})
buttons = buttons.find_all("span",{"class":"channel-name"})
channel_names = []
for button in buttons:
    channel_names.append(button.text.replace('\n',""))
    
for i in range(1, len(channel_names[1:])):
    channel_names[i] = channel_names[i] .replace("+", "")
    channel_names[i] = channel_names[i] .replace("&", "_")
    channel_names[i] = channel_names[i] .replace("-", "") 
print(len(channel_names))

825


In [7]:
search_urls = []
links = []
base_url ="https://mi.tv"
for i in channel_names[1:]:
    search_link = 'https://mi.tv/br/pesquisar/'+i
    search_urls.append(search_link)
    
print(len(search_urls))

for i in search_urls:    
    driver.get(i)
    page_source =driver.page_source
    soup = BeautifulSoup(page_source,'html.parser')
    channel_links = soup.find("ul", {"class":"channels"})
    if channel_links !=None:
        channel_links = channel_links.find_all("li")
        for channel_link in channel_links:
            tv_link = channel_link.find("a").get("href")
            tv_link = base_url+tv_link
            if tv_link not in links:
                links.append(tv_link)

824


slink = "https://mi.tv/br/canais/amc-hd/2022-09-01"
driver.get(slink)
page_source =driver.page_source

class Data:
    def __init__(self, link, date):
        self.driver = webdriver.Chrome(ChromeDriverManager().install())
        self.link = link+"/"+date

    def get_data(self):
        from datetime import datetime
        self.driver.get(self.link)
        page_source =self.driver.page_source
        soup = BeautifulSoup(page_source,'html.parser')
        try:
            channel_detail = soup.find("div", {"class":"channel-info"})
        except:
            channel_detail = None
        try:
            channel_name = channel_detail.find("h1").text.replace('\n',"")
        except:
            channel_name = None
        try:
            program_date = channel_detail.find("span").text.replace('\n',"")
        except:
            program_date = None
        try:
            programs = soup.find("ul", {"class":"broadcasts time24"})
        except:
            programs  =None
        try:
            programs = programs.find_all("li")
        except:
            programs  =None
        for program in programs:
            content = program.find("div", {"class":"content"})
            try:
                prog_link = program.find("a", {"class":"program-link"}).get("href")   
            except:
                prog_link = None
            try:
                prog_name = content.find("h2").text.replace('\n',"")
                prog_name = re.sub(r"[\t]*", "", prog_name)
            except:
                prog_name = None
                
            try:
                prog_time = content.find("span", {"class":"time"}).text.replace('\n',"")
            except:
                prog_time = None
            try:
                prog_genre = content.find("span", {"class":"sub-title"}).text.replace('\n',"")
                prog_genre = re.sub(r"[\t]*", "", prog_genre)
            except:
                prog_genre = None
            try:
                prog_synopsis = content.find("p", {"class":"synopsis"}).text.replace('\n',"")
                prog_synopsis = re.sub(r"[\t]*", "", prog_synopsis)
            except:
                prog_synopsis = None
                
            try:
                now = datetime.now()
                date_scraped = now.strftime("%d/%m/%Y %H:%M:%S")
            except:
                date_scraped = None
            
            tv = {"channel":channel_name, "program name":prog_name, "program time":prog_time,"program genre":prog_genre, "program synopsis":prog_synopsis, "program link":prog_link,  "date":program_date, "scraped link":self.link, "scrape date":date_scraped}
            
            return tv
    

In [ ]:
len(links)

In [ ]:
from datetime import datetime
data = []
for link in links:
    driver.get(link)
    page_source =driver.page_source
    soup = BeautifulSoup(page_source,'html.parser')
    
    programs = soup.find("ul", {"class":"broadcasts time24"})
    if programs != None:
        programs = programs.find_all("li")
        for program in programs:
            content = program.find("div", {"class":"content"})
            channel_detail = soup.find("div", {"class":"channel-info"})
            try:
                channel_name = channel_detail.find("h1").text.replace('\n',"")
            except:
                channel_name = None
            try:
                program_date = channel_detail.find("span").text.replace('\n',"")
            except:
                program_date = None
            try:
                prog_link = program.find("a", {"class":"program-link"}).get("href")   
            except:
                prog_link = None
            try:
                prog_name = content.find("h2").text.replace('\n',"")
                prog_name = re.sub(r"[\t]*", "", prog_name)
            except:
                prog_name = None
                        
            try:
                prog_time = content.find("span", {"class":"time"}).text.replace('\n',"")
            except:
                prog_time = None
            try:
                prog_genre = content.find("span", {"class":"sub-title"}).text.replace('\n',"")
                prog_genre = re.sub(r"[\t]*", "", prog_genre)
            except:
                prog_genre = None
            try:
                prog_synopsis = content.find("p", {"class":"synopsis"}).text.replace('\n',"")
                prog_synopsis = re.sub(r"[\t]*", "", prog_synopsis)
            except:
                prog_synopsis = None
                        
            try:
                now = datetime.now()
                date_scraped = now.strftime("%d/%m/%Y %H:%M:%S")
            except:
                date_scraped = None
                    
            tv = {"channel":channel_name, "program name":prog_name, "program time":prog_time,"program genre":prog_genre, "program synopsis":prog_synopsis, "program link":prog_link,  "date":program_date, "scraped link":link, "scrape date":date_scraped}
                    
            data.append(tv)
    

In [ ]:
###For a single date use above
df = pd.DataFrame.from_dict(data)
df.head()

## For  a range of Dates

In [9]:
from datetime import date, timedelta
start_date = date(2022, 8, 2) 
end_date = date(2022, 9, 2)    # perhaps date.now()

def get_range_links(links, start_date, end_date):
    dates = []
    scrape_links = []
    from datetime import date, timedelta
    delta = end_date - start_date   # returns timedelta
    for i in links:
        for j in range(delta.days + 1):
            day = start_date + timedelta(days=j)
            day = day.strftime('%Y-%m-%d')
            scrape_links.append(i+"/"+day)
    
    return scrape_links
        

In [11]:
from datetime import date, timedelta

start_date = date(2022, 8, 2) 
end_date = date(2022, 9, 2) 
scrape_links = get_range_links(links, start_date, end_date)
scrape_links
len(links)

811

In [20]:
def get_data(scrape_links):
    from datetime import datetime
    scrape_links = scrape_links
    data = []
    for link in scrape_links:
        driver.get(link)
        page_source =driver.page_source
        soup = BeautifulSoup(page_source,'html.parser')
        
        programs = soup.find("ul", {"class":"broadcasts time24"})
        if programs != None:
            programs = programs.find_all("li")
            for program in programs:
                content = program.find("div", {"class":"content"})
                channel_detail = soup.find("div", {"class":"channel-info"})
                try:
                    channel_name = channel_detail.find("h1").text.replace('\n',"")
                except:
                    channel_name = None
                try:
                    program_date = channel_detail.find("span").text.replace('\n',"")
                except:
                    program_date = None
                try:
                    prog_link = program.find("a", {"class":"program-link"}).get("href")   
                except:
                    prog_link = None
                try:
                    prog_name = content.find("h2").text.replace('\n',"")
                    prog_name = re.sub(r"[\t]*", "", prog_name)
                except:
                    prog_name = None
                            
                try:
                    prog_time = content.find("span", {"class":"time"}).text.replace('\n',"")
                except:
                    prog_time = None
                try:
                    prog_genre = content.find("span", {"class":"sub-title"}).text.replace('\n',"")
                    prog_genre = re.sub(r"[\t]*", "", prog_genre)
                except:
                    prog_genre = None
                try:
                    prog_synopsis = content.find("p", {"class":"synopsis"}).text.replace('\n',"")
                    prog_synopsis = re.sub(r"[\t]*", "", prog_synopsis)
                except:
                    prog_synopsis = None
                            
                try:
                    now = datetime.now()
                    date_scraped = now.strftime("%d/%m/%Y %H:%M:%S")
                except:
                    date_scraped = None
                        
                tv = {"channel":channel_name, "program name":prog_name, "program time":prog_time,"program genre":prog_genre, "program synopsis":prog_synopsis, "program link":prog_link,  "date":program_date, "scraped link":link, "scrape date":date_scraped}
                        
                data.append(tv)
                
    return data
    

In [21]:
data_range = get_data(scrape_links = get_range_links(links, start_date, end_date))
data_range

[{'channel': 'Programação + Globosat Terça  2 de agosto',
  'program name': 'Mundo Visto de Cima',
  'program time': '06:00',
  'program genre': 'Temporada 1 Episódio 2 - Estados Unidos',
  'program synopsis': 'Viajamos do norte do Rio Hudson até o coração da cidade de Nova York. O percurso inclui atrações como a lendária Academia Militar de West Point e a cidade cenário da história do Cavaleiro Sem Cabeça.',
  'program link': '/br/programas/mundo-visto-de-cima-s01e02-rio-hudson-nova-york',
  'date': 'Terça  2 de agosto',
  'scraped link': 'https://mi.tv/br/canais/globosat/2022-08-02',
  'scrape date': '03/09/2022 08:14:12'},
 {'channel': 'Programação + Globosat Terça  2 de agosto',
  'program name': None,
  'program time': None,
  'program genre': None,
  'program synopsis': None,
  'program link': None,
  'date': 'Terça  2 de agosto',
  'scraped link': 'https://mi.tv/br/canais/globosat/2022-08-02',
  'scrape date': '03/09/2022 08:14:12'},
 {'channel': 'Programação + Globosat Terça  2

In [24]:
df_range = pd.DataFrame.from_dict(data_range)
df_range

,channel,program name,program time,program genre,program synopsis,program link,date,scraped link,scrape date
0,Programação + Globosat Terça 2 de agosto,Mundo Visto de Cima,06:00,Temporada 1 Episódio 2 - Estados Unidos,Viajamos do norte do Rio Hudson até o coração ...,/br/programas/mundo-visto-de-cima-s01e02-rio-h...,Terça 2 de agosto,https://mi.tv/br/canais/globosat/2022-08-02,03/09/2022 08:14:12
1,Programação + Globosat Terça 2 de agosto,None,None,None,None,None,Terça 2 de agosto,https://mi.tv/br/canais/globosat/2022-08-02,03/09/2022 08:14:12
2,Programação + Globosat Terça 2 de agosto,Mundo Visto de Cima,06:30,Temporada 1 Episódio 3 - Dinamarca,"Uma viagem pela Dinamarca, país com mais de 48...",/br/programas/mundo-visto-de-cima-s01e03-dinam...,Terça 2 de agosto,https://mi.tv/br/canais/globosat/2022-08-02,03/09/2022 08:14:12
3,Programação + Globosat Terça 2 de agosto,Hotéis Incríveis,07:00,"Temporada 7 Episódio 18 - Canto das Águas, Pou...","Começamos esse episódio na Bahia, Lençóis, com...",/br/programas/hoteis-incriveis-s07e18-canto-da...,Terça 2 de agosto,https://mi.tv/br/canais/globosat/2022-08-02,03/09/2022 08:14:12
4,Programação + Globosat Terça 2 de agosto,Ruas Mundo Afora,07:30,Temporada 1 Episódio 36 - Londres,Rodrigo Ruas cruza o mundo em busca de experiê...,/br/programas/ruas-mundo-afora-s01e36-londres,Terça 2 de agosto,https://mi.tv/br/canais/globosat/2022-08-02,03/09/2022 08:14:12
...,...,...,...,...,...,...,...,...,...
335,Programação + Globosat Segunda 8 de agosto,Ruas Mundo Afora,03:00,Temporada 1 Episódio 40 - Irlanda,Rodrigo Ruas cruza o mundo em busca de experiê...,/br/programas/ruas-mundo-afora-s01e40-irlanda,Segunda 8 de agosto,https://mi.tv/br/canais/globosat/2022-08-08,03/09/2022 08:15:18
336,Programação + Globosat Segunda 8 de agosto,Brasil Visto de Cima,03:30,Temporada 3 Episódio 13 - Litoral Sul do ES,Embarque em um emocionante voo pelo Brasil. Po...,/br/programas/brasil-visto-de-cima-s03e13-lito...,Segunda 8 de agosto,https://mi.tv/br/canais/globosat/2022-08-08,03/09/2022 08:15:18
337,Programação + Globosat Segunda 8 de agosto,Destino ao Alvo,04:00,Temporada 2 Episódio 1 - Colômbia,"Conheça os lugares onde Pablo Escobar viveu, f...",/br/programas/destino-ao-alvo-s02e01-colombia,Segunda 8 de agosto,https://mi.tv/br/canais/globosat/2022-08-08,03/09/2022 08:15:18
338,Programação + Globosat Segunda 8 de agosto,Destino ao Alvo,04:30,Temporada 2 Episódio 9 - Curaçao,Sorin passa um tempo nas antigas Antilhas Neer...,/br/programas/destino-ao-alvo-s02e09-curacao,Segunda 8 de agosto,https://mi.tv/br/canais/globosat/2022-08-08,03/09/2022 08:15:18


In [25]:
df.to_csv("mitv_2022_09_01.csv")

In [26]:
df_range.to_csv("mi_tv_range.csv")